# Question 1

In [1]:
def python_script(mydata):
    
    #Data
    mydata.to_csv("mydata.tbl", sep='\t')
    
    #R Script
    with open("myfunction.r", "w") as out:
        out.write("library(edgeR) \n")
        out.write("infile     <- 'mydata.tbl' \n" )
        out.write("group      <- factor(c(1,1,1,2,2,2)) \n")   
        out.write("outfile    <- 'myresult.out' \n")

        out.write("x     <- read.table(infile, sep='\t', row.names=1) \n")
        out.write("y     <- DGEList(counts=x,group=group) \n")
        out.write("y     <- estimateDisp(y) \n")
        out.write("et    <- exactTest(y) \n")
        out.write("tab   <- topTags(et, nrow(x))\n")

        out.write("write.table(tab, file=outfile)\n")
    !Rscript myfunction.r
    
    #Output table 
    tables = pd.read_table("myresult.out", delim_whitespace = True)
    return(tables)
    
    


# Question 2

In [2]:
import pandas as pd
data1 = pd.read_table("w08-data.1", index_col=0)
data2 = pd.read_table("w08-data.2", index_col = 0)
data3 = pd.read_table("w08-data.3", index_col = 0)


#Merged datasets
table1 = data1.merge(data2, left_index=True, right_index = True)
table2 = data2.merge(data3, left_index=True, right_index = True)
table3 = data1.merge(data3, left_index=True, right_index = True)



In [3]:
#Python script result tables
tables12 = python_script(table1)
tables23 = python_script(table2)
tables13 = python_script(table3)



Loading required package: limma
Design matrix not provided. Switch to the classic mode.
Loading required package: limma
Design matrix not provided. Switch to the classic mode.
Loading required package: limma
Design matrix not provided. Switch to the classic mode.


In [4]:
print(tables12[tables12 < .05].count().PValue)
print(tables23[tables23 < .05].count().PValue)
print(tables13[tables13 < .05].count().PValue)

2135
996
2049


He combined tables 1 and 2, as can be seen above by 2135 p values below .05. The second table (table23) has the fewest number of low p-values. This result suggests that table 2 and 3 are similar, meaning data tables 2 and 3 are both wild type samples. Table 1 is the mutant sample. Further support for this analysis is that 996 - the number of false positives - is ~5% of 20,000 (total genes). This is the amount of false positives we would expect under a true null hypothesis (e.g. both wild type groups). 

# Question 3

No, I do not agree. He failed to account for multiple comparisons. There are 200031 comparison tests being run, which increases the probability of type I error. To correct for this, the alpha level must be adjusted. Thus we use the Bonferroni correction, which is most conservative. We also provide a less conservative estimate in the form of FDR through the BH correction.

https://xkcd.com/882/

If we have a family of hypothesis $H_1...H_n$, the Bonferroni correction controls the family wise error rate (FWER). The FWER is the probability of rejecting at least one true $H_i$ - making atleast one type 1 error. The Bonferroni correction rejects the null hypothesis for each p value $p_{i} \leq {\frac {\alpha }{n}}$,  controlling the FWER at $ \leq \alpha$. 

In [5]:
print(tables12[tables12 < .05/(20031)].count().PValue)

51


We could also do a BH correction and look for FDR below .05 which is less conservative 

In [6]:
print(tables12[tables12 < .05 ].count().FDR)

69


The Bonferonni correction comes at the cost of increased false negatives, which reduces statistical power, but offers the least type 1 error. Thus I will take the most conservative number to reduce type 1 error probability and conclude ~51 genes are differentially expressed. 

# Question 4

I will normalize RNA seq both within and between samples, and filter.

In [7]:
def python_script2(mydata):
    mydata.to_csv("mydata.tbl", sep='\t')
            
    with open("myfunction.r", "w") as out:
        out.write("library(edgeR) \n")
        out.write("infile     <- 'mydata.tbl' \n" )
        out.write("group      <- factor(c(1,1,1,2,2,2)) \n")   
        out.write("outfile    <- 'myresult.out' \n")

        out.write("x     <- read.table(infile, sep='\t', row.names=1) \n")
        out.write("y     <- DGEList(counts=x,group=group) \n")
        
        #Filter low-expression genes, setting a cpm requirment of 10 in atleast 2 samples. 
        out.write(" keep <- rowSums(cpm(y)>10) >= 2 \n")
        out.write("print(sum(keep == 'TRUE')) \n")
        out.write("y <- y[keep, , keep.lib.sizes=FALSE]\n")
        
        #Normalize within and between samples 
        out.write("y$samples$lib.size <- colSums(y$counts) \n ")
        out.write("y     <- calcNormFactors(y, method='TMM') \n")
        
        out.write("y     <- estimateCommonDisp(y) \n")
        out.write("et    <- exactTest(y) \n")
        out.write("tab   <- topTags(et, nrow(x))\n")

        out.write("write.table(tab, file=outfile)\n")

    !Rscript myfunction.r
    tables = pd.read_table("myresult.out", delim_whitespace = True)
    return(tables)
    
tables12_p = python_script2(table1)


Loading required package: limma
[1] 16554


In [8]:
print(tables12_p[tables12_p < .05].count().FDR)
print(tables12_p[tables12_p < .05/(16554)].count().PValue)

52
50


Again both corrections are used, and the less conservative BH correction more closely approaches the Bonferonni correction under this corrected edgeR function. I will again use the Bonferonni threshold to conclude 50 genes are differnetially expressed. The problem was that Moriarty's script did not remove low infuence points, and did not normalize. From edgeR documentation:

"As a rule of thumb, genes are dropped if they can’t possibly be expressed in all the samples for
any of the conditions. Users can set their own definition of genes being expressed. Usually a gene
is required to have a count of 5-10 in a library to be considered expressed in that library."
